# Running Full Model

In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

from preprocess.data_preprocessing import *
from features.featuring import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California_10.json.gz"
meta_url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz"

cleaned_df = get_single_chunk(url,meta_url) #get_clean_review_data(url, meta_url)
featured_df = featuring_engineering(cleaned_df)

Loading metadata from: /Users/kevinb/Desktop/cse158/RDSB/rsdb/data/metadata.json.gz
Loaded 463034 metadata entries.
Processing review data from: /Users/kevinb/Desktop/cse158/RDSB/rsdb/data/data.json.gz
Processed 79763 entries in the first chunk.


In [40]:
from models.tlfm.temporal_dynamic_v import TemporalDynamicVariants
from models.fpmc.fpmc_v import FPMCVariants

In [41]:
featured_df

,review_time(unix),reviewer_id,gmap_id,rating,isin_category_restaurant,isin_category_park,isin_category_store,lon_bin_0,lon_bin_1,lon_bin_2,...,lat_bin_15,lat_bin_16,lat_bin_17,lat_bin_18,lat_bin_19,closed_on_weekend,weekly_operating_hours,time_bin,user_mean_time,prev_item_id
76001,1.322523,1.000046e+20,0x8091aad7a7c4d8f3:0xe4aa0b5a0d2fbcb8,5,0,0,0,0,0,0,...,0,0,0,0,0,False,47.0,2676141,0.634746,0x8090504ec98781ad:0xd5235eff697582d1
83574,0.394244,1.000331e+20,0x808fcecafdb31371:0xd9acab442d78ae57,5,0,0,0,0,0,0,...,0,0,0,0,0,False,75.0,2598092,0.029116,0x808fcecafdb31371:0x8bd0cb4a080e362f
12856,1.226621,1.000378e+20,0x80c333fe0f143baf:0xd4c005c08bb117d6,2,0,0,0,0,0,0,...,0,0,0,0,0,False,40.0,2668078,1.160554,0x80dcac8730afa353:0x51a1d5eebf63f15f
98068,-0.507204,1.000410e+20,0x80c3281188d0d8a7:0xd9055ec15e626693,5,0,0,1,0,0,0,...,0,0,0,0,0,False,91.0,2522298,-0.539851,0x80dcad120946ced7:0x61926b580ff47c0a
13096,0.926559,1.000463e+20,0x80db57d6f3f45b17:0xa2494325a8e241cf,5,0,0,0,0,0,0,...,0,0,0,0,0,False,77.0,2642849,0.722403,0x80db41130e809e9b:0x96166498b19c5e97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64884,-0.147489,1.184288e+20,0x80c2b66db72fe81b:0xfd5e2f86736d05f4,5,0,0,0,0,0,0,...,0,0,0,0,0,False,70.0,2552542,-0.194477,0x80dd33c48f82119f:0xace1cb276e60af3e
103333,1.006453,1.184296e+20,0x80c32fa16dbdf329:0xf8cbe9d0e8fb186c,4,0,0,0,0,0,0,...,0,0,0,0,0,False,76.0,2649566,0.369517,0x80c32789715c74dd:0x9c332f98ca8c42cc
14720,-0.291660,1.184301e+20,0x80c2d39636c5d87f:0x3d868d3def90a396,5,0,0,0,0,0,0,...,0,0,0,0,0,False,58.0,2540421,-0.523079,0x80c2cf91a4492b67:0xf36a895f62d7c8f5
30486,0.282890,1.184347e+20,0x80c36440129b0c55:0x94d1998653c8f3e9,4,0,0,0,0,0,0,...,0,0,0,0,0,False,37.0,2588729,-0.044054,0x80c365d97d417abf:0xdeb65caeafecadfe


***
# Prepare Features
***

In [42]:
df = featured_df
data_query = df[['gmap_id', 'reviewer_id', 'rating']]
df.info()

# need sequential?
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6157 entries, 76001 to 73346
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   review_time(unix)         6157 non-null   float64
 1   reviewer_id               6157 non-null   float64
 2   gmap_id                   6157 non-null   object 
 3   rating                    6157 non-null   int64  
 4   isin_category_restaurant  6157 non-null   int64  
 5   isin_category_park        6157 non-null   int64  
 6   isin_category_store       6157 non-null   int64  
 7   lon_bin_0                 6157 non-null   int64  
 8   lon_bin_1                 6157 non-null   int64  
 9   lon_bin_2                 6157 non-null   int64  
 10  lon_bin_3                 6157 non-null   int64  
 11  lon_bin_4                 6157 non-null   int64  
 12  lon_bin_5                 6157 non-null   int64  
 13  lon_bin_6                 6157 non-null   int64  
 14  lon

In [43]:
def tlfm_df_to_tf_dataset(dataframe):
    '''change featuers from data frame to tensorfloe styles'''
    
    return tf.data.Dataset.from_tensor_slices({
        "reviewer_id": dataframe["reviewer_id"].astype(str),
        "gmap_id": dataframe["gmap_id"].astype(str),
        "time": dataframe["review_time(unix)"].astype(float),
        "time_bin": dataframe["time_bin"].astype(float),
        "user_mean_time": dataframe["user_mean_time"],
        "rating": dataframe["rating"],
        "isin_category_restaurant": dataframe["isin_category_restaurant"].astype(float),
        "isin_category_park": dataframe["isin_category_park"].astype(float),
        "isin_category_store": dataframe["isin_category_store"].astype(float),
        "closed_on_weekend": dataframe["closed_on_weekend"].astype(float),
        "weekly_operating_hours": dataframe["weekly_operating_hours"].astype(float),
        
        # Longitude bins
        **{f"lon_bin_{i}": dataframe[f"lon_bin_{i}"].astype(float) for i in range(20) if f"lon_bin_{i}" in dataframe.columns},
        # Latitude bins
        **{f"lat_bin_{i}": dataframe[f"lat_bin_{i}"].astype(float) for i in range(20) if f"lat_bin_{i}" in dataframe.columns},
    })

def fpmc_df_to_tf_dataset(dataframe):
    '''change featuers from data frame to tensorfloe styles'''
    dataframe["reviewer_id"] = dataframe["reviewer_id"].astype(str)
    dataframe["prev_item_id"] = dataframe["prev_item_id"].astype(str)
    dataframe["gmap_id"] = dataframe["gmap_id"].astype(str)
    
    user_lookup = tf.keras.layers.StringLookup(
        vocabulary=dataframe["reviewer_id"].unique(), mask_token=None
    )
    item_lookup = tf.keras.layers.StringLookup(
        vocabulary=dataframe["gmap_id"].unique(), mask_token=None
    )
    
    return tf.data.Dataset.from_tensor_slices({
        "reviewer_id": user_lookup(dataframe["reviewer_id"]),
        "prev_item_id": item_lookup(dataframe["prev_item_id"]),
        "next_item_id": item_lookup(dataframe["gmap_id"]),
        "rating": dataframe["rating"].astype(float),
        "isin_category_restaurant": dataframe["isin_category_restaurant"].astype(float),
        "isin_category_park": dataframe["isin_category_park"].astype(float),
        "isin_category_store": dataframe["isin_category_store"].astype(float),
        "closed_on_weekend": dataframe["closed_on_weekend"].astype(float),
        "weekly_operating_hours": dataframe["weekly_operating_hours"].astype(float),
        
        # Longitude bins
        **{f"lon_bin_{i}": dataframe[f"lon_bin_{i}"].astype(float) for i in range(20) if f"lon_bin_{i}" in dataframe.columns},
        # Latitude bins
        **{f"lat_bin_{i}": dataframe[f"lat_bin_{i}"].astype(float) for i in range(20) if f"lat_bin_{i}" in dataframe.columns},
    })


***
# Description (Many Assumptions)
***

## Temporal Latent Factor Model + Neural Corrolative (Dynamic Latent)
**Notice that item $i$ refers to the business and user $u$ refers to the user.**

**Modification**:

$$
\gamma_{u,k}(t) = \gamma_{u,k} + \alpha_{u,k} \cdot \text{dev}_u(t) + \gamma_{u,k,t}
$$

$$
\hat{r}_{u,i,t} = \mu + b_i + b_i(t) + b_u + \alpha_u \cdot \text{dev}_u(t) + f(\gamma_{u,k}(t), \gamma_{i,k})
$$

**Optimization**:

$$
\arg \min_{\alpha, \beta, \gamma} \sum_{u,i} \left(\mu + b_i + b_i(t) + b_u + \alpha_u \cdot \text{dev}_u(t) + f(\gamma_{u,k}(t), \gamma_{i,k}) - R_{u,i} \right)^2 + \lambda \left[ \sum_u \beta_u^2 + \sum_i \beta_i^2 + \sum_i \left\| \gamma_i \right\|_2^2 + \sum_u \left\| \gamma_u \right\|_2^2 \right]
$$

In [46]:
train_data = tlfm_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
test_data = tlfm_df_to_tf_dataset(test_df).batch(4096)

In [50]:
embedding_dim = 30
dense_units = 30
l2_reg = 0.0201
time_bins= 30
model = TemporalDynamicVariants(l2_reg, dense_units, embedding_dim, data_query, time_bins)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_root_mean_squared_error", 
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2, 
    decay_steps=1000, 
    decay_rate=0.8
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule))
model.fit(train_data, epochs=500, validation_data=test_data, callbacks=[early_stopping])

Epoch 1/500
2/2 [==============================] - 1s 176ms/step - root_mean_squared_error: 1.9695 - loss: 6.0855 - regularization_loss: 2.8041 - total_loss: 8.8895 - val_root_mean_squared_error: 1.7108 - val_loss: 2.9268 - val_regularization_loss: 0.9764 - val_total_loss: 3.9033
Epoch 2/500
2/2 [==============================] - 0s 19ms/step - root_mean_squared_error: 1.6537 - loss: 2.2738 - regularization_loss: 0.6035 - total_loss: 2.8773 - val_root_mean_squared_error: 1.9259 - val_loss: 3.7090 - val_regularization_loss: 0.2627 - val_total_loss: 3.9718
Epoch 3/500
2/2 [==============================] - 0s 23ms/step - root_mean_squared_error: 1.9794 - loss: 3.8685 - regularization_loss: 0.3052 - total_loss: 4.1737 - val_root_mean_squared_error: 1.3102 - val_loss: 1.7166 - val_regularization_loss: 0.4734 - val_total_loss: 2.1900
Epoch 4/500
2/2 [==============================] - 0s 21ms/step - root_mean_squared_error: 1.3272 - loss: 1.7198 - regularization_loss: 0.5653 - total_loss: 2.

In [51]:
test_metrics = model.evaluate(test_data, return_dict=True)
print(f"Test RMSE: {test_metrics['root_mean_squared_error']}")

predictions = []
actual_ratings = []
for batch in test_data:
    predicted_ratings = model(batch).numpy()
    actual_ratings.extend(batch["rating"].numpy())
    predictions.extend(predicted_ratings)

predictions = np.array(predictions)
actual_ratings = np.array(actual_ratings)

rmse = np.sqrt(np.mean((predictions - actual_ratings) ** 2))
print(f"RECHECK RMSE: {rmse}")

if np.all(actual_ratings == actual_ratings.round()):
    correct = np.mean(predictions.round() == actual_ratings)
    print(f"Rounded Accuracy: {correct}")
else:
    print("Actual ratings are not integers, skipping rounded accuracy calculation.")

1/1 [==============================] - 0s 17ms/step - root_mean_squared_error: 1.1442 - loss: 1.3091 - regularization_loss: 0.0301 - total_loss: 1.3392
Test RMSE: 1.1441680192947388
RECHECK RMSE: 1.1441680726347163
Rounded Accuracy: 0.27213647441104794


***
# Description (Less Assumptions)
***

# Factorized Personalized Markov Chain

$$
p(i_{t+1} \mid i_t, u) = p(i_{t+1} \mid i_t, u)
$$

In Factorized Personalized Markov Chain (FPMC), we do this calculation by:

$$
f(i \mid u, j) = \underbrace{\gamma_{ui} \cdot \gamma_{iu}}_{\mathclap{f(i \mid u)}} + \underbrace{\gamma_{ij} \cdot \gamma_{ji}}_{\mathclap{f(i \mid j)}} + \underbrace{\gamma_{uj} \cdot \gamma_{ju}}_{\mathclap{f(u, j)}}.
$$

Neglecting independent terms:

$$
f(i \mid u, j) = 
\underbrace{\gamma_{ui} \cdot \gamma_{iu}}_{\text{user's compatibility with the next item}} + 
\underbrace{\gamma_{ij} \cdot \gamma_{ji}}_{\text{next item's compatibility with the previous item}}
$$

In [31]:
train_data = fpmc_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
test_data = fpmc_df_to_tf_dataset(test_df).batch(4096)

In [37]:
embedding_dim = 32
l2_reg = 0.0201
lr = 1e-3
model = FPMCVariants(l2_reg=l2_reg, embedding_dim=embedding_dim, data_query=data_query)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_root_mean_squared_error", 
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

history = model.fit(
    train_data, 
    validation_data=test_data, 
    epochs=500, 
    callbacks=[early_stopping]
)

Epoch 1/200
2/2 [==============================] - 1s 244ms/step - root_mean_squared_error: 5.0045 - loss: 24.7556 - regularization_loss: 6.7647 - total_loss: 31.5203 - val_root_mean_squared_error: 4.8007 - val_loss: 23.0472 - val_regularization_loss: 6.2987 - val_total_loss: 29.3459
Epoch 2/200
2/2 [==============================] - 0s 21ms/step - root_mean_squared_error: 4.8493 - loss: 23.2759 - regularization_loss: 6.0773 - total_loss: 29.3533 - val_root_mean_squared_error: 4.6489 - val_loss: 21.6119 - val_regularization_loss: 5.6492 - val_total_loss: 27.2611
Epoch 3/200
2/2 [==============================] - 0s 21ms/step - root_mean_squared_error: 4.6971 - loss: 22.3145 - regularization_loss: 5.4466 - total_loss: 27.7611 - val_root_mean_squared_error: 4.4988 - val_loss: 20.2396 - val_regularization_loss: 5.0553 - val_total_loss: 25.2950
Epoch 4/200
2/2 [==============================] - 0s 23ms/step - root_mean_squared_error: 4.5470 - loss: 20.1227 - regularization_loss: 4.8709 - t

In [38]:
test_metrics = model.evaluate(test_data, return_dict=True)
print(f"Test RMSE: {test_metrics['root_mean_squared_error']}")

predictions = []
actual_ratings = []
for batch in test_data:
    predicted_ratings = model(batch).numpy()
    actual_ratings.extend(batch["rating"].numpy())
    predictions.extend(predicted_ratings)

predictions = np.array(predictions)
actual_ratings = np.array(actual_ratings)

rmse = np.sqrt(np.mean((predictions - actual_ratings) ** 2))
print(f"RECHECK RMSE: {rmse}")

if np.all(actual_ratings == actual_ratings.round()):
    correct = np.mean(predictions.round() == actual_ratings)
    print(f"Rounded Accuracy: {correct}")
else:
    print("Actual ratings are not integers, skipping rounded accuracy calculation.")

1/1 [==============================] - 0s 17ms/step - root_mean_squared_error: 1.1812 - loss: 1.3952 - regularization_loss: 0.1502 - total_loss: 1.5454
Test RMSE: 1.1811875104904175
RECHECK RMSE: 1.1811874576036936
Rounded Accuracy: 0.31844029244516653
